# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones

Mentoría: Análisis de sentimiento sobre noticias financieras e indicadores financieros

Grupo B

Integrantes:

Alonso, Guillermo

Ramos, Pablo Nicolás

Rocco, Victor

Salama, Ana

In [1]:
# Se importan librerías
import sklearn.neighbors._base 
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

from xgbimputer import XGBImputer

from MissingValuesHandler.missing_data_handler import RandomForestImputer

from datetime import datetime

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, plot_confusion_matrix, roc_auc_score, roc_curve

from sklearn import tree
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

from lightgbm import LGBMClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import cross_val_score, cross_validate, KFold 
from sklearn.model_selection import StratifiedKFold, LeaveOneOut, LeavePOut
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score, confusion_matrix
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from joblib import dump

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import confusion_matrix

from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from missingpy import MissForest

sns.set_context('talk')

import plotly
plotly.__version__

np.set_printoptions(suppress=True)  # no usar notacion "e"

random_seed = 42

In [2]:
# Se lee el dataset que se obtuvo en la segunda entrega
df = pd.read_csv(os.path.join(os.getcwd(), "data", "clean_news_dataset_v2.csv"))

In [3]:
# Inspeccionamos los datos
df.shape

(221513, 12)

In [4]:
# Fill empty and NaNs values with NaN
dataset = df.fillna(np.nan)

# Check for Null values
dataset.isnull().sum()

id                   0
ticker               0
title                0
category             0
content              8
release_date         0
provider             0
url                  0
article_id           0
release_date_year    0
clean_content        8
clean_title          4
dtype: int64

In [5]:
# Revisamos si hay registros duplicados
df.duplicated().any()

False

In [6]:
df = df.dropna()

In [8]:
df.dtypes

id                    int64
ticker               object
title                object
category             object
content              object
release_date         object
provider             object
url                  object
article_id            int64
release_date_year     int64
clean_content        object
clean_title          object
dtype: object

In [9]:
df.isnull().sum()

id                   0
ticker               0
title                0
category             0
content              0
release_date         0
provider             0
url                  0
article_id           0
release_date_year    0
clean_content        0
clean_title          0
dtype: int64

In [10]:
df.head()

,id,ticker,title,category,content,release_date,provider,url,article_id,release_date_year,clean_content,clean_title
0,221515,NIO,Why Shares of Chinese Electric Car Maker NIO A...,news,What s happening\nShares of Chinese electric c...,2020-01-15,The Motley Fool,https://invst.ly/pigqi,2060327,2020,happening share chinese electric car maker nio...,share chinese electric car maker nio flying hi...
1,221516,NIO,NIO only consumer gainer Workhorse Group amon...,news,Gainers NIO NYSE NIO 7 \nLosers MGP Ingr...,2020-01-18,Seeking Alpha,https://invst.ly/pje9c,2062196,2020,gainer nio nyse nio loser mgp ingredient nasda...,nio consumer gainer workhorse group among loser
2,221517,NIO,NIO leads consumer gainers Beyond Meat and Ma...,news,Gainers NIO NYSE NIO 14 Village Farms In...,2020-01-15,Seeking Alpha,https://invst.ly/pifmv,2060249,2020,gainer nio nyse nio village farm international...,nio lead consumer gainer beyond meat mattel am...
3,221518,NIO,NIO NVAX among premarket gainers,news,Cemtrex NASDAQ CETX 85 after FY results \n...,2020-01-15,Seeking Alpha,https://invst.ly/picu8,2060039,2020,cemtrex nasdaq cetx fy result fluent nasdaq fl...,nio nvax among premarket gainer
4,221519,NIO,PLUG NIO among premarket gainers,news,aTyr Pharma NASDAQ LIFE 63 on Kyorin Pharm...,2020-01-06,Seeking Alpha,https://seekingalpha.com/news/3529772-plug-nio...,2053096,2020,atyr pharma nasdaq life kyorin pharma deal tow...,plug nio among premarket gainer


In [11]:
df.category.value_counts()

opinion    125928
news        95573
Name: category, dtype: int64